In [ ]:
spark

## Services
- [Catalog](http://localhost:8181/)
- [Storage](http://localhost:9001/)

## DDL

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS demo.nyc;

In [ ]:
%%sql
CREATE TABLE demo.nyc.taxis (
    VendorID              bigint,
    tpep_pickup_datetime  timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count       double,
    trip_distance         double,
    RatecodeID            double,
    store_and_fwd_flag    string,
    PULocationID          bigint,
    DOLocationID          bigint,
    payment_type          bigint,
    fare_amount           double,
    extra                 double,
    mta_tax               double,
    tip_amount            double,
    tolls_amount          double,
    improvement_surcharge double,
    total_amount          double,
    congestion_surcharge  double,
    airport_fee           double
)
TBLPROPERTIES ('format-version'=3, 'write.parquet.compression-codec'='zstd');

In [ ]:
%%sql
CREATE TEMPORARY VIEW nyc_taxis_yello_tripdata
USING org.apache.spark.sql.parquet
OPTIONS (
  path "/home/iceberg/data/yellow_tripdata_2022-01.parquet"
);

In [ ]:
%%sql
INSERT INTO demo.nyc.taxis SELECT * FROM nyc_taxis_yello_tripdata;

## DQL

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis LIMIT 10;

In [ ]:
%%sql
SELECT Count(*) FROM demo.nyc.taxis;

In [ ]:
%%sql
SELECT VendorId, Count(*) FROM demo.nyc.taxis GROUP BY VendorID;

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis WHERE VendorID = 2 LIMIT 10;

In [ ]:
%%sql
EXPLAIN SELECT * FROM demo.nyc.taxis WHERE VendorID = 2 LIMIT 10;

## DML

In [ ]:
%%sql --limit 10
SELECT * FROM demo.nyc.taxis WHERE VendorID = 5

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.history;

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.snapshots;

In [ ]:
%%sql
UPDATE demo.nyc.taxis
SET passenger_count = 1
WHERE VendorID = 5

## Time Travel

In [ ]:
%%sql --limit 10
SELECT * FROM demo.nyc.taxis VERSION AS OF 506386827648522500 WHERE VendorID = 5

In [ ]:
%%sql --limit 10
SELECT * FROM demo.nyc.taxis WHERE VendorID = 5

## Metadata Tables

https://iceberg.apache.org/docs/nightly/spark-queries/#inspecting-tables

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.manifests;

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.history;

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.snapshots;

In [ ]:
%%sql --limit 1
SELECT * FROM demo.nyc.taxis.files

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.refs;

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.partitions;

In [ ]:
%%sql --limit 5
SELECT * FROM demo.nyc.taxis.position_deletes;

## Branching

https://iceberg.apache.org/docs/nightly/branching/

In [ ]:
%%sql
ALTER TABLE demo.nyc.taxis CREATE BRANCH `demo_branch`

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.refs;

In [ ]:
%%sql
ALTER TABLE demo.nyc.taxis SET TBLPROPERTIES (
    'write.update.mode'='merge-on-read'
);

In [ ]:
%%sql
UPDATE demo.nyc.taxis
SET passenger_count = 2
WHERE VendorID = 5

In [ ]:
%%sql --limit 10
SELECT * FROM demo.nyc.taxis WHERE VendorID = 5

In [ ]:
%%sql --limit 10
SELECT * FROM demo.nyc.taxis VERSION AS OF 'demo_branch' WHERE VendorID = 5

## Views

https://iceberg.apache.org/docs/nightly/spark-ddl/#iceberg-views-in-spark

In [ ]:
%%sql
CREATE VIEW demo.nyc.taxis_cash_payment
AS
SELECT * FROM demo.nyc.taxis
WHERE payment_type = 2

In [ ]:
%%sql
SELECT VendorId, ROUND(SUM(total_amount), 2) FROM demo.nyc.taxis_cash_payment
GROUP BY VendorId

In [ ]:
%%sql
EXPLAIN SELECT VendorId, ROUND(SUM(total_amount), 2) FROM demo.nyc.taxis_cash_payment
GROUP BY VendorId

## Partitioning

- https://iceberg.apache.org/docs/nightly/partitioning/
- https://iceberg.apache.org/spec/#partition-transforms

In [ ]:
%%sql
ALTER TABLE demo.nyc.taxis ADD PARTITION FIELD year(tpep_pickup_datetime)

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.partitions;

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW nyc_taxis_yello_tripdata
USING org.apache.spark.sql.parquet
OPTIONS (
  path "/home/iceberg/data/yellow_tripdata_2021-12.parquet"
);

In [ ]:
%%sql
INSERT INTO demo.nyc.taxis SELECT * FROM nyc_taxis_yello_tripdata;

In [ ]:
%%sql
SELECT * FROM demo.nyc.taxis.partitions;

In [ ]:
%%sql
SELECT Year(tpep_pickup_datetime), Month(tpep_pickup_datetime), Count(*)
FROM demo.nyc.taxis
GROUP BY Year(tpep_pickup_datetime), Month(tpep_pickup_datetime)

In [ ]:
%%sql --limit 10
SELECT *
FROM demo.nyc.taxis
WHERE Date(tpep_pickup_datetime) = '2021-12-31'

In [ ]:
%%sql
EXPLAIN
SELECT *
FROM demo.nyc.taxis
WHERE Date(tpep_pickup_datetime) = '2021-12-31'

## Misc / Optimizations

In [ ]:
%%sql
CALL demo.system.compute_partition_stats('nyc.taxis')

In [ ]:
%%sql
CALL demo.system.compute_table_stats('nyc.taxis')

In [ ]:
%%sql
CALL demo.system.rewrite_data_files(table => 'nyc.taxis', strategy => 'sort', sort_order => 'zorder(VendorID,payment_type)');